In [5]:
import numpy as np
%matplotlib qt
import matplotlib.pyplot as pl
import time
import pickle
import os

In [23]:
class State:
    def __init__(self,L:int,p:float,q:int,initial_state) -> None:
        assert L%2 == 0
        self.p = p
        self.q = q
        self.L = L
        # index = np.arange(0,2**L,1)
        self.data = initial_state.copy()
        self.log_Z = []
        
        self.data = np.zeros((2,)*L)
        
     

In [15]:
def eventransfer(State,U_list : list):
    L = State.L
    if len(U_list) == 1:
        U_list = U_list*(L//2)
    os = State.data.reshape((4,)*(L//2))
    for x in range(L//2):
        U = U_list[x]
        os = np.tensordot(U,os,axes=(-1,x))
        os = np.moveaxis()
    os = np.reshape(os,(2,)*L)
    State.data = os


def oddtransfer(State,U_list : list):
    L = State.L
    if len(U_list) == 1:
        U_list = U_list*(L//2)
    os = State.data
    os = np.moveaxis(os,0,-1) #bringing 1st spin to the last position
    os = np.reshape(os,(4,)*(L//2))

    for x in range(L//2):
        U = U_list[x]
        os = np.tensordot(U,os,axes=(-1,x))
    os = np.reshape(os,(2,)*L)
    os = np.moveaxis(os,-1,0)
    State.data = os

In [12]:
def no_of_down_spins(N):
    temp = np.arange(0,2**N,1,dtype=int)
    no_of_ones = np.zeros(2**N,dtype=float)
    for i in range(N):
        no_of_ones += temp%2
        temp = (temp/2).astype(int)
    return N - no_of_ones

def TopLayer(state,no_of_downs):
    """
    This function contracts the evolved state with the top layer.
    """

    ## Caluclate no. of down spins in each configuration
    # no_of_downs = no_of_down_spins(state.L)
    fac = np.sum(state.data.reshape(2**state.L)*((state.q)**(no_of_downs-(state.L)//2)))
    
    return np.log(fac)

In [13]:
def get_U_haar(q):
    U = np.zeros((4,4))
    U[0,0] = 1
    U[3,0] = 0
    U[3,3] = 1
    U[0,3] = 0
    U[3,1] = q/(q**2+1)
    U[3,2] = q/(q**2+1)
    U[0,1] = q/(q**2+1)
    U[0,2] = q/(q**2+1)
    return U

def get_U_t(p,q):
    U = np.zeros((4,4))
    U[0,0] = p**2
    U[3,0] = (1-p**2)/q**2
    U[3,3] = 1
    U[0,3] = 0
    U[3,1] = q*p**2/(q**2+1) + (1-p**2)/q
    U[3,2] = q*p**2/(q**2+1) + (1-p**2)/q
    U[0,1] = p**2*q/(q**2+1)
    U[0,2] = p**2*q/(q**2+1)
    return U

def get_U_k(p,q):
    U = np.zeros((4,4))
    U[0,0] = 1
    U[3,0] = 0
    U[3,3] = p**2
    U[0,3] = (1-p**2)/q**2
    U[0,2] = q*p**2/(q**2+1) + (1-p**2)/q
    U[0,1] = q*p**2/(q**2+1) + (1-p**2)/q
    U[3,2] = p**2*q/(q**2+1)
    U[3,1] = p**2*q/(q**2+1)
    return U

In [28]:
def encoded_bit_state(L,depth,q,BC='up'): 
    """
    This function evolves a single LOCAL bell pair to global bit by evolving by Haar dynamics. BC specify whether the initial boundary condition of the bell pair is 'up' or 'down' corresponding to Identity or Swap permutation respectively.
    """
    # up and down correspond to bottom boundary condition for the bell pair.
    
    initial_state = np.zeros((2,)*L)
    if BC == 'up':
        initial_state[1,:] = 1
    elif BC == 'down':
        initial_state[0,:] = 1
    else:
        print("BC parameter is wrong. It can only take 'up' or 'down' value")
    
    state = State(L=L,p=None,q=q,initial_state=initial_state)
    log_Z = []
     # Scrambling
    U_haar = get_U_haar(q)
    for t in range(depth):
        start = time.time()
        if t%2 == 0:
            eventransfer(state,[U_haar])
        else:
            oddtransfer(state,[U_haar])
        sd = np.sum(state.data)
        log_Z.append(np.log(sd))
        state.data = state.data/sd
    state.log_Z.append(np.sum(log_Z)) # storing the Z (partition function) for the encoding process

    return state


def load_bit_state(L,depth,q,BC='up'):
    filedir = 'data/encoded_bell_pairs'
    if not os.path.isdir(filedir):
        os.makedirs(filedir)
    
    filename = filedir + '/L='+str(L)+'_T='+str(depth)+'_q='+str(q)+'_'+BC
    if os.path.isfile(filename):
        with open(filename,'rb') as f:
            state = pickle.load(f)
        return state
    
    state = encoded_bit_state(L,depth,q,BC)
    with open(filename,'wb') as f:
        pickle.dump(state,f)
    
    return state
    

In [29]:
def get_U_list(U_t,U_k,U_haar,a_string):
    U_list = []
    N = len(a_string)
    if a_string == 't'*N:
        U_list = [U_t]
    elif a_string == 'k'*N:
        U_list = [U_k]
    elif a_string == 'h'*N:
        U_list = [U_haar]
    
    else:
        for x in range(N):
            if a_string[x] == 't':
                U_list.append(U_t)
            elif a_string[x] == 'k':
                U_list.append(U_k)
            elif a_string[x] == 'h':
                U_list.append(U_haar)
    
    return U_list

## function to get free energy when the strength of coupling between S and ancillas is same everywhere
def free_energy_uniform(state: State, depth,ancilla_string : list):
    """
    ancilla_string is a list whose elements are string of length L//2. For the element 't','k','h', the transfer matrix corresponding to having the ancilla traced out, kept in system, no ancilla respectively are applied. 
    """
    p = state.p
    q = state.q
    L = state.L

    top_layer_factor = []
    ##
    U_t = get_U_t(p,q)
    U_k = get_U_k(p,q)
    U_haar = get_U_haar(q)

    no_of_downs = no_of_down_spins(L)

    for t in range(depth):
        # start = time.time()
        U_list = get_U_list(U_t,U_k,U_haar,ancilla_string[t])
        if t%2 == 0:
            eventransfer(state,U_list)
        else:
            oddtransfer(state,U_list)
        sd = np.sum(state.data)
        state.log_Z.append(np.log(sd))
        state.data = state.data/sd
        # print(time.time()-start)
        top_layer_factor.append(TopLayer(state,no_of_downs))
    

    return state,top_layer_factor


In [30]:
def get_ancilla_string(L,T):
    a_string = ['t'*(L//2) for i in range(T)]
    return a_string

In [46]:
## Calculate coherent QI for a single encoded bell pair.
F = {}
F_t = {}
for L in [8][:]:
    p_list = np.round(np.linspace(0.5,1,10),2)
    T = 2*L
    q = 2
    F[L] = []
    F_t[L] = {}
    for p in p_list:
        F_t[L][p] = []
        state_up = load_bit_state(L,2*L,q,'up')
        state_down = load_bit_state(L,2*L,q,'down')
        state_up.p = p
        state_down.p = p
        a_string = get_ancilla_string(L,T)
        print(state_down.log_Z)
        state_up,top_layer_up= free_energy_uniform(state_up,T,a_string)
        state_down, top_layer_down= free_energy_uniform(state_down,T,a_string)

        print(a_string,len(state_down.log_Z),top_layer_up)
        for t in range(T):
            temp_up =  top_layer_up[t]
            temp_down =  top_layer_down[t]
            F_t[L][p].append(temp_down-temp_up)
        F[L].append(F_t[L][p][-1])
            

[nan]
['tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt'] 17 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan]
['tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt'] 17 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan]
['tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt'] 17 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan]
['tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt'] 17 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan]
['tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt', 'tttt

##### Check how single qubit scrambling helps?

In [42]:
for L in [8]:
    for p in F_t[L]:
        print
        T_data = np.arange(1,len(F_t[L][p])+1,1)[::2]/L
        print(F_t[L][p])
        pl.plot(T_data,np.array(F_t[L][p])[::2]/(np.log(q)),'-o',label = r'$p=$'+str(p))
pl.legend()

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [130]:
for L in F:
    pl.plot(p_list,[np.array(F_t[L][p][-1])/np.log(q) for p in p_list],'-o',label = r'$L=$'+str(L))

pl.ylabel(r'$\Delta F$',fontsize=16)
pl.xlabel(r'$p$',fontsize=16)

pl.title(r'Haar random dynamics for single Bell pair,'+'\n'+ '$q=2,T=10L$, $p_O=p_E=0.5$',fontsize=16)

pl.legend(fontsize=16)
pl.tight_layout()

In [242]:
p_list

array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])